In [110]:
import pandas as pd
import sqlite3 as db
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score,explained_variance_score

In [2]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'
db_name = 'trailheadDirectBirds_sous.db'

In [3]:
##connect to database
def connectDB():
    try:
        cnx = db.connect(db_name)
    except Exception as cnxError:
        raise UserWarning(f'Unable to connect to database due to: {cnxError}')
    return cnx

In [4]:
def stopQualityMask(speciesCode: str,closestStop: str):
    cnx = connectDB()
    try:
        gap = dt.date.today().year-2018
        query = f'SELECT COUNT(DISTINCT(year)) as "frq" FROM coefficients_bySpecies WHERE speciesCode = "{speciesCode}" AND closestStop = "{closestStop}"'
        coeficients = pd.read_sql(query,con=cnx)
        coeficients['frq'] = coeficients.apply(lambda g: (g.frq/gap),axis=1)
    except Exception as maskExc:
        raise maskExc
    return coeficients['frq']

For birds with robust data, do they behave the same at all stops? If not, what stops does each species appear to prefer? Is that preference explained by habitat?

In [147]:
def wklyAbd_selectSpecies(cnx,speciesList: list):
    cnx=cnx
    try:
        querySpecies = []
        for i in speciesList:
            i = str(i)
            querySpecies.append(i)
        querySpecies = str(querySpecies).strip('[]')
        #query = f'SELECT speciesCode,FX.locId,StopName,obsDt,howMany FROM historicObservations AS FX  LEFT JOIN closestStop on FX.locId = closestStop.locId WHERE (SELECT count(distinct(subId)) FROM historicObservations AS QA WHERE QA.comName = FX.comName) > 2 AND FX.locId in ({query_locIds})'
        ##todo #83 write an exception to convert non-string speciesList values to strings that returns the db.error if the strings do not represent columns --DONE
        query = f'SELECT speciesCode,FX.locId,StopName,obsDt,howMany FROM historicObservations AS FX  LEFT JOIN closestStop on FX.locId = closestStop.locId WHERE (SELECT count(distinct(subId)) FROM historicObservations AS QA WHERE QA.comName = FX.comName) > 2 AND FX.speciesCode in ({querySpecies});'
        obsData = pd.read_sql(query,con=cnx,parse_dates=['obsDt'])
        obsData = obsData.assign(obsDt_week=obsData.obsDt.dt.isocalendar().week)
        obsData['howMany'].fillna(1,inplace=True)
        obsData = obsData.groupby(['speciesCode','StopName','obsDt_week'])['howMany'].mean().reset_index()
        #maxCount = obsData.groupby(['speciesCode'])['howMany'].max()
        avgCount = obsData.groupby(['speciesCode'])['howMany'].mean()
        #derive relative abundance
        obsData['relativeAbundance'] = obsData.apply(lambda x: (x.howMany/(avgCount[x.speciesCode])),axis=1)  #baseline relative abd, same process as Fink et. all
        obsData['relativeAbundance'] = obsData.apply(lambda f: stopQualityMask(f.speciesCode,f.StopName)*f.relativeAbundance,axis=1)  #apply frequency mask
        avgAbd = obsData.groupby(['speciesCode'])['relativeAbundance'].mean()
        obsData['relativeAbundance'] = obsData.apply(lambda n: ((n.relativeAbundance)/(avgAbd[n.speciesCode])),axis=1)  #normalizing around average relative abundance
        obsData.sort_values(by=['StopName','obsDt_week'],ascending=True,inplace=True)

                #apply absence smoothing mask
        container = []
        speciesContainer = []
        stopKeys = obsData.drop_duplicates(subset=['StopName'])
        for x in speciesList:
            for StopName in stopKeys.itertuples():
                stop_obsData = obsData[obsData['speciesCode']==x]
                stop_obsData = stop_obsData[stop_obsData['StopName']==StopName.StopName]
                allweek = pd.DataFrame({'obsDt_week':range(1,53)})
                stop_obsData.drop(columns=['speciesCode','StopName','howMany'],inplace=True)
                stop_obsData = pd.merge(left=stop_obsData,right=allweek,left_on='obsDt_week',right_on='obsDt_week',how='outer')
                stop_obsData.set_index('obsDt_week',inplace=True)
                stop_obsData.sort_index(axis='index',ascending=True,inplace=True)
                stop_obsData['mask'] = stop_obsData['relativeAbundance'].interpolate(method='ffill',limit=5,limit_direction='forward')  #mask, values do not matter
                stop_obsData['mask'] = stop_obsData['mask'].interpolate(method='bfill',limit=2,limit_direction='backward')  #mask, values do not matter
                stop_obsData.loc[stop_obsData['mask'].isna() == True,'relativeAbundance'] = 0
                stop_obsData['fx_relativeAbundance'] = stop_obsData['relativeAbundance'].interpolate(method='linear',limit=5,limit_direction='forward')
                stop_obsData['fx_relativeAbundance'] = stop_obsData['fx_relativeAbundance'].interpolate(method='linear',limit=2,limit_direction='backward')
                stop_obsData.drop(columns=['relativeAbundance','mask'],inplace=True)
                stop_obsData.rename(columns={'fx_relativeAbundance':StopName.StopName},inplace=True)
                #stop_obsData.reset_index(inplace=True)
                #stop_obsData['StopName'] = StopName.StopName
                #stop_obsData['speciesCode'] = x
                #stop_obsData.drop(columns=['mask'],inplace=True)
                
                container.append(stop_obsData)
            weeklySpeciesAbd = pd.concat(container,ignore_index=False,axis=1)
            weeklySpeciesAbd = weeklySpeciesAbd.fillna(value=0,axis=0)
            weeklySpeciesAbd['speciesCode'] = x
            weeklySpeciesAbd.reset_index(inplace=True)
            speciesContainer.append(weeklySpeciesAbd)
        popularSpecies = pd.DataFrame(pd.concat(speciesContainer,ignore_index=True,axis=0))

    except db.DatabaseError as dbExc:
        raise f'there was an issue with the database request: {dbExc}'
    except Exception as ex:
        raise ex
    finally: cnx.close()
    return popularSpecies

In [148]:
amerob = wklyAbd_selectSpecies(cnx=connectDB(),speciesList=['gockin'])

In [140]:
fit = [(stop,r2_score(amerob.MountTeneriffe,amerob[stop])) for stop in list(amerob)]
var = [(stop,explained_variance_score(amerob.MountTeneriffe,amerob[stop])) for stop in list(amerob)]

#fit_df = pd.DataFrame(data=[fit,var],columns=['stopName','r_squared','explainedVariance'])
#fit_df.sort_values(by='r_squared',ascending=False,inplace=True)
#fit_df['r_squared'] = fit_df['r_squared'].astype('float64')
#fit_df['error'] = 1-fit_df['r_squared']
#fit_df = fit_df[fit_df['r_squared'] > 0.65]
fit

[('ChiricoTrail_PooPooPoint', -28.046615005680504),
 ('EastSunsetWay', -2.978924254005184),
 ('HighSchool', 0.32057768626443184),
 ('MargaretsWay', -32.254608348031894),
 ('MountTeneriffe', 1.0),
 ('SquakMountain', -1.8049838199225543)]

In [61]:
selection = amerob[amerob['StopName']=='MountTeneriffe']
series = pd.Series(name='MountTeneriffe',data=selection['fx_relativeAbundance'],index=range(1,53))
series

1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
10   NaN
11   NaN
12   NaN
13   NaN
14   NaN
15   NaN
16   NaN
17   NaN
18   NaN
19   NaN
20   NaN
21   NaN
22   NaN
23   NaN
24   NaN
25   NaN
26   NaN
27   NaN
28   NaN
29   NaN
30   NaN
31   NaN
32   NaN
33   NaN
34   NaN
35   NaN
36   NaN
37   NaN
38   NaN
39   NaN
40   NaN
41   NaN
42   NaN
43   NaN
44   NaN
45   NaN
46   NaN
47   NaN
48   NaN
49   NaN
50   NaN
51   NaN
52   NaN
Name: MountTeneriffe, dtype: float64

In [ ]:


        #apply absence smoothing mask
        container = []
        
        stopKeys = obsData.drop_duplicates(subset=['StopName'])
        for StopName in stopKeys.itertuples():
            stop_obsData = obsData[obsData['speciesCode']==stopKeys.StopName]

            ##removing this block, it corrupts interpolation comparisons
            #weekMin = (species_obsData['obsDt_week'].min()-8)
            #if weekMin <= 0: weekMin = 1
            #weekMax = (species_obsData['obsDt_week'].max()+8)
            #if weekMax > 53: weekMax = 53
            #allweek = pd.DataFrame({'obsDt_week':range(weekMin,weekMax)})
            
            allweek = pd.DataFrame({'obsDt_week':range(1,53)})
            stop_obsData.drop(columns=['speciesCode','StopName'],inplace=True)
            stop_obsData = pd.merge(left=stop_obsData,right=allweek,left_on='obsDt_week',right_on='obsDt_week',how='outer')
            stop_obsData.set_index('obsDt_week',inplace=True)
            stop_obsData.sort_index(axis='index',ascending=True,inplace=True)
            stop_obsData['mask'] = stop_obsData['relativeAbundance'].interpolate(method='index',limit=5,limit_direction='both')  #mask, values do not matter
            stop_obsData.loc[stop_obsData['mask'].isna() == True,'relativeAbundance'] = 0
            stop_obsData.reset_index(inplace=True)
            stop_obsData['StopName'] = StopName.StopName
            stop_obsData.drop(columns=['mask'],inplace=True)
            container.append(stop_obsData)
        weeklySpeciesAbd = pd.DataFrame(pd.concat(list,ignore_index=True))
    except db.DatabaseError as dbExc:
        raise f'there was an issue with the database request: {dbExc}'
    except Exception as ex:
        raise ex
    finally: cnx.close()
    return weeklySpeciesAbd